In [24]:
# Import Block
from rdkit import Chem
from openbabel import pybel
from openff.toolkit.topology import Molecule,Topology
from openff.toolkit.utils import RDKitToolkitWrapper
import openmm
from simtk.openmm import app
from openmm import unit
from openmm.app import PDBFile
from openff.toolkit.typing.engines.smirnoff import ForceField


In [6]:
# Wrapper and FF setup
# Use RDKit wrapper
rdktkw = RDKitToolkitWrapper()

# Loading setup parameters
forcefield = ForceField('openff_unconstrained-1.1.0.offxml')

In [10]:
# load pdb with one copy of pdb file
off_mol = Molecule.from_pdb_and_smiles('7101899.pdb', "CC1=CN=C(C(=C1OC)C)C[S@@](=O)C2=NC3=C(N2)C=C(C=C3)OC")
# load supercell pdb file (2x2x2) into topology
pdb_file = PDBFile('7101899_supercell.pdb')
off_top = Topology.from_openmm(pdb_file.topology, [off_mol])
# Create MD simulation inputs
system = forcefield.create_openmm_system(off_top)
integrator = openmm.VerletIntegrator(1*unit.femtoseconds)
platform = openmm.Platform.getPlatformByName('Reference')

/home/qualenal/anaconda3/envs/openFF/lib/python3.9/site-packages/openmm/app/internal/pdbstructure.py:537: UserWarning: WARNING: duplicate atom (HETATM    2  N   UNL     1       2.401   2.650   6.162  1.00  0.00           N  , HETATM    1  N   UNL     1       3.871   1.194   5.274  1.00  0.00           N  )
  warnings.warn("WARNING: duplicate atom (%s, %s)" % (atom, old_atom._pdb_string(old_atom.serial_number, atom.alternate_location_indicator)))
/home/qualenal/anaconda3/envs/openFF/lib/python3.9/site-packages/openmm/app/internal/pdbstructure.py:537: UserWarning: WARNING: duplicate atom (HETATM    3  N   UNL     1      -0.835  -0.054   3.095  1.00  0.00           N  , HETATM    2  N   UNL     1       2.401   2.650   6.162  1.00  0.00           N  )
  warnings.warn("WARNING: duplicate atom (%s, %s)" % (atom, old_atom._pdb_string(old_atom.serial_number, atom.alternate_location_indicator)))
/home/qualenal/anaconda3/envs/openFF/lib/python3.9/site-packages/openmm/app/internal/pdbstructure.py

In [11]:
# create simulation
simulation = openmm.app.Simulation(pdb_file.topology, system, integrator, platform)
# set initial positions from pdbfile
positions = pdb_file.getPositions()
simulation.context.setPositions(positions)

In [ ]:
# set reporters
pdb_reporter = openmm.app.PDBReporter('trajectory.pdb', 1)
state_data_reporter = openmm.app.StateDataReporter(
    "data.csv",
    1,
    step=True,
    potentialEnergy=True,
    temperature=True,
    density=True,
)
simulation.reporters.append(pdb_reporter)
simulation.reporters.append(state_data_reporter)

In [12]:
simulation.saveState('initial')
orig_potential = simulation.context.getState(getEnergy=True).getPotentialEnergy()
#state_data_reporter.report(simulation, simulation.context.getState())

print('Minimizing Energy!')
simulation.minimizeEnergy()
min_state = simulation.context.getState(getEnergy=True, getPositions=True)

Minimizing Energy!


In [14]:
print(orig_potential)
print(min_state.getPotentialEnergy())

15797.24736288079 kJ/mol
4742.126017499885 kJ/mol


In [22]:
ref_mol = off_mol.to_rdkit()
min_coords = min_state.getPositions()
print(min_coords)

[Vec3(x=0.44766535710535155, y=0.1499465224120548, z=0.5452254673853929), Vec3(x=0.2597415034610252, y=0.2762152129866017, z=0.5713164415073497), Vec3(x=-0.10479555221240734, y=0.011559097384119471, z=0.3412237049344834), Vec3(x=0.3093007896721612, y=0.16241346287537234, z=0.5242952523001156), Vec3(x=0.3662437599135615, y=0.3436325413373122, z=0.6228487681247356), Vec3(x=0.3714144489952728, y=0.47145847514287065, z=0.6753559488155296), Vec3(x=0.4950776957957809, y=0.5183797013591958, z=0.7227964528247003), Vec3(x=0.6101971005098644, y=0.43768048606402904, z=0.7198790291628195), Vec3(x=0.6009826054130649, y=0.3100257715311563, z=0.6605531147213584), Vec3(x=0.47813491380987166, y=0.2671677185949682, z=0.6115379595336788), Vec3(x=0.10475512896443158, y=0.1284959485983752, z=0.32119170084581083), Vec3(x=0.0019636174675996695, y=0.03496059433960728, z=0.2650047487268734), Vec3(x=0.01594134574202208, y=-0.026165235612957225, z=0.13822472954633122), Vec3(x=-0.08507675950251492, y=-0.113970011

In [21]:
simulation.step(1)


In [27]:
import pymol
pymol.cmd.load('7101899_supercell.pdb','initial')
pymol.cmd.load('trajectory.pdb','final')
rmsd = pymol.cmd.align('final','initial',cycles=0)
print(rmsd[0])


1.3957754373550415
